In [ ]:
# train_and_export.py  — Checkpoints (10k), Resume, Größen ins Manifest,
# OOM-freundliche Hi-Res-Inferenz per Tiling, schlanke Logs/Plots

import copy
import logging
import random
import sys
import json
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional

import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib import colors
from torch import nn
from tqdm import tqdm
from tqdm.contrib.logging import logging_redirect_tqdm

# ---- Headless-Backend: kein Tkinter nötig ----
matplotlib.use('Agg')
plt.rcParams['font.family'] = 'cmr10'
plt.rcParams['mathtext.fontset'] = 'cm'

DPI: int = 100
SCALE: float = 2
GR = (1 + np.sqrt(5)) / 2 - 1

GRAY: plt.Colormap = colors.LinearSegmentedColormap.from_list('gray', plt.get_cmap('gray')(np.linspace(0, 1., 100)))
SEISMIC: plt.Colormap = colors.LinearSegmentedColormap.from_list('seismic', plt.get_cmap('seismic')(np.linspace(0, 1., 100)))
SEISMIC_NEGATIVE: plt.Colormap = colors.LinearSegmentedColormap.from_list('seismic_neg', plt.get_cmap('seismic')(np.linspace(0., .5, 50)))
SEISMIC_POSITIVE: plt.Colormap = colors.LinearSegmentedColormap.from_list('seismic_pos', plt.get_cmap('seismic')(np.linspace(.5, 1., 50)))

# ==== AUSGABE-ORTE (Jupyter-/Script-freundlich) ====
ROOT: Path = Path.cwd()              # funktioniert auch in Jupyter
OUTPUT_DIR: Path = ROOT / 'output'
IMAGE_DIR: Path = ROOT / 'images'
MODELS_DIR: Path = ROOT / 'webroot' / 'models'

# ---- Auto-Discovery: alle Bilder in images/ (ohne Endung) als Projekte ----
ALLOWED_EXTS = ['.png', '.jpg', '.jpeg', '.bmp', '.webp']

def discover_projects(image_dir: Path) -> List[str]:
    if not image_dir.exists():
        image_dir.mkdir(parents=True, exist_ok=True)
        return []
    projects = []
    for p in image_dir.iterdir():
        if p.is_file() and p.suffix.lower() in ALLOWED_EXTS:
            projects.append(p.stem)
    projects.sort()
    return projects

AUTO_DISCOVER_PROJECTS = True
PROJECTS: List[str] = discover_projects(IMAGE_DIR) if AUTO_DISCOVER_PROJECTS else [
    'mona-lisa_1080', 'girl_1080', 'nebelmeer_1080', 'schrei_1080', 'sterne_1080'
]

SIZE = 256  # Hidden width und Inputbild-Skalierung
CHECKPOINT_EVERY = 10_000

def save_fig(fig: plt.Figure, path: Path, dpi: float = DPI) -> None:
    if path is not None:
        path.parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(path, format=path.suffix[1:], transparent=False, dpi=dpi)

def save_image(im: np.ndarray, path: Path) -> None:
    im = im.astype(np.int32)
    im[im > 255] = 255
    im[im < 0] = 0
    fig = plt.Figure(figsize=(im.shape[1], im.shape[0]), dpi=1)
    sub = fig.add_subplot()
    sub.set_axis_off()
    sub.imshow(im[:, :, ::-1], interpolation='nearest')
    fig.subplots_adjust(bottom=0, top=1, left=0, right=1)
    sub.margins(0, 0)
    save_fig(fig, path, 1)

def save_art(art: np.ndarray, path: Path) -> None:
    w, b = np.nanmin(art), np.nanmax(art)
    art = art.copy()
    art -= w
    if (b - w) != 0:
        art *= 255 / (b - w)
    im = art.astype(np.int32)
    im[im > 255] = 255
    im[im < 0] = 0
    fig = plt.Figure(figsize=(im.shape[1], im.shape[0]), dpi=1)
    sub = fig.add_subplot()
    sub.set_axis_off()
    sub.imshow(im[:, :, ::-1], interpolation='nearest', zorder=1)
    sub.set_xlim(0, im.shape[1])
    sub.set_ylim(0, im.shape[0])
    fig.subplots_adjust(bottom=0, top=1, left=0, right=1)
    sub.margins(0, 0)
    save_fig(fig, path, 1)

def save_frame(fig: plt.Figure, n: int, model: torch.nn.Module, im: np.ndarray, art: np.ndarray, losses: List[float]) -> None:
    weights = [w.detach().cpu().numpy() for i, w in model.named_parameters() if 'weight' in i][1:-1]
    im = im.astype(np.int32)
    im[im > 255] = 255
    im[im < 0] = 0
    art = art.copy()
    w, b = np.nanmin(art), np.nanmax(art)
    art -= w
    if (b - w) != 0:
        art *= 255 / (b - w)
    art = art.astype(np.int32)
    fig.clear()
    sub = fig.add_subplot(2, 1, 2)
    sub.set_axis_off()
    sub.set_yscale('log', base=10)
    sub.plot(losses[-5_000:], c='r')
    sub = fig.add_subplot(2, len(weights) + 2, len(weights) + 1)
    sub.set_axis_off()
    sub.imshow(im[:, :, ::-1], interpolation='nearest', zorder=1)
    sub = fig.add_subplot(2, len(weights) + 2, len(weights) + 2)
    sub.set_axis_off()
    sub.imshow(art[:, :, ::-1], interpolation='nearest', zorder=1)
    sub.set_xlim(0, art.shape[1])
    sub.set_ylim(0, art.shape[0])
    vmin, vmax = np.nanmin(weights), np.nanmax(weights)
    if vmin < 0 < vmax:
        norm = colors.TwoSlopeNorm(vmin=vmin, vcenter=0, vmax=vmax)
        cmap = SEISMIC
    elif vmax == vmin:
        norm = colors.Normalize(vmin=vmin, vmax=vmax)
        cmap = SEISMIC
    elif vmax < 0:
        norm = colors.Normalize(vmin=vmin, vmax=0)
        cmap = SEISMIC_NEGATIVE
    else:
        norm = colors.Normalize(vmin=0, vmax=vmax)
        cmap = SEISMIC_POSITIVE
    for i, w in enumerate(weights):
        sub = fig.add_subplot(2, len(weights) + 2, i + 1)
        sub.set_axis_off()
        sub.set_frame_on(True)
        sub.imshow(w, cmap=cmap, norm=norm, interpolation='nearest', zorder=1)
    fig.suptitle(f'{n}\n({np.min(losses):.3f})', fontsize=14)
    fig.subplots_adjust(bottom=.1, top=.9, left=0.02, right=.98, wspace=.05, hspace=.05)
    fig.canvas.flush_events()

def artsy(weights: List[torch.Tensor], biases: List[torch.Tensor]) -> Tuple[List[torch.Tensor], List[torch.Tensor]]:
    weights = [weights[0]] + [weights[3].rot90(), weights[2].rot90().rot90(), weights[1].rot90().rot90().rot90()] + [weights[4]]
    biases = [biases[0]] + [biases[3], biases[2], biases[1]] + [biases[4]]
    return weights, biases

def find_image_for_project(project: str, image_dir: Path) -> Path:
    candidates = [image_dir / f"{project}{ext}" for ext in ALLOWED_EXTS]
    for p in candidates:
        if p.exists():
            return p
    existing = sorted([str(p.name) for p in image_dir.glob('*') if p.is_file()])
    raise FileNotFoundError(
        f"Kein Bild für Projekt '{project}' gefunden.\n"
        f"Erwartet: {', '.join([project+e for e in ALLOWED_EXTS])} in {image_dir}\n"
        f"Aktuell vorhanden ({len(existing)}): {existing}"
    )

def read_and_resize_image(project: str, size: int, image_dir: Path) -> np.ndarray:
    path = find_image_for_project(project, image_dir)
    logging.info(f'Loading {path}')
    im = cv2.imread(str(path), cv2.IMREAD_COLOR)
    if im is None:
        raise RuntimeError(f"cv2 konnte die Datei nicht lesen: {path}")
    if im.shape[0] > im.shape[1]:
        im = cv2.resize(im, (round(size / im.shape[0] * im.shape[1]), size))
    else:
        im = cv2.resize(im, (size, round(size / im.shape[1] * im.shape[0])))
    return im

# ========= Speicher-schonende Inferenz (gekachelt) =========

def render_image_tiled(model: nn.Module, H: int, W: int, device: torch.device, tile: int = 256) -> np.ndarray:
    out = np.zeros((H, W, 3), dtype=np.float32)
    model.eval()
    with torch.inference_mode():
        for y0 in range(0, H, tile):
            y1 = min(H, y0 + tile)
            yy, xx = np.mgrid[y0:y1, 0:W]
            x_in = np.stack([
                yy.reshape(-1) / max(1, (H - 1)),
                xx.reshape(-1) / max(1, (W - 1)),
            ], axis=1).astype(np.float32)
            x_t = torch.from_numpy(x_in).to(device, non_blocking=True)
            pred = model(x_t).detach().cpu().numpy().reshape(y1 - y0, W, 3)
            out[y0:y1, :, :] = pred
            del x_t, pred, x_in, yy, xx
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    return out

def render_art_tiled(model: nn.Module, H: int, W: int, device: torch.device, tile: int = 256) -> np.ndarray:
    art = copy.deepcopy(model)
    ws, bs = [], []
    for name, p in art.named_parameters():
        if 'weight' in name: ws.append(p.detach())
        elif 'bias' in name: bs.append(p.detach())
    ws, bs = artsy(ws, bs)
    for name, p in art.named_parameters():
        if 'weight' in name: p.data = nn.parameter.Parameter(ws.pop(0))
        elif 'bias' in name: p.data = nn.parameter.Parameter(bs.pop(0))
    art = art.to(device).eval()

    out = np.zeros((H, W, 3), dtype=np.float32)
    with torch.inference_mode():
        for y0 in range(0, H, tile):
            y1 = min(H, y0 + tile)
            yy, xx = np.mgrid[y0:y1, 0:W]
            x_in = np.stack([
                yy.reshape(-1) / max(1, (H - 1)),
                xx.reshape(-1) / max(1, (W - 1)),
            ], axis=1).astype(np.float32)
            x_t = torch.from_numpy(x_in).to(device, non_blocking=True)
            pred = art(x_t).detach().cpu().numpy().reshape(y1 - y0, W, 3)
            out[y0:y1, :, :] = pred
            del x_t, pred, x_in, yy, xx
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
    del art, ws, bs
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return out

# ========= Checkpointing & Export =========

def _tensor_to_list(t: torch.Tensor) -> List[List[float]] | List[float]:
    return t.detach().cpu().numpy().tolist()

def build_layers_from_state_dict(state: Dict[str, torch.Tensor]) -> List[Dict[str, Any]]:
    items = [(int(k.split('.')[0]), k, v) for k, v in state.items() if k.endswith('.weight') or k.endswith('.bias')]
    items.sort(key=lambda x: (x[0], 0 if x[1].endswith('.weight') else 1))
    pairs: List[tuple[torch.Tensor, torch.Tensor]] = []
    i = 0
    while i < len(items):
        idx, key, val = items[i]
        if key.endswith('.weight'):
            bkey = f"{idx}.bias"
            bias = None
            for j in range(i + 1, len(items)):
                if items[j][1] == bkey:
                    bias = items[j][2]
                    break
            if bias is None:
                raise RuntimeError(f"Bias für {key} nicht gefunden")
            pairs.append((val, bias))
        i += 1
    layers: List[Dict[str, Any]] = []
    for li, (W, b) in enumerate(pairs):
        layers.append({"W": _tensor_to_list(W), "b": _tensor_to_list(b)})
        if li < len(pairs) - 1:
            layers.append({"act": "tanh"})
    return layers

def save_model_json(model: nn.Module, project_name: str, shape: Tuple[int, int, int], models_dir: Path = MODELS_DIR) -> Path:
    models_dir.mkdir(parents=True, exist_ok=True)
    state = model.state_dict()
    payload = {
        "name": project_name,
        "shape": {"height": int(shape[0]), "width": int(shape[1]), "channels": int(shape[2])},
        "layers": build_layers_from_state_dict(state)
    }
    out_path = models_dir / f"{project_name}.json"
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    logging.info(f"Model-JSON geschrieben: {out_path}")
    return out_path

def write_manifest(models_dir: Path = MODELS_DIR) -> Path:
    models_dir.mkdir(parents=True, exist_ok=True)
    files = sorted([p for p in models_dir.glob("*.json") if p.name != "manifest.json"])
    manifest = []
    for p in files:
        try:
            with p.open("r", encoding="utf-8") as f:
                data = json.load(f)
            shape = data.get("shape", {})
            manifest.append({
                "name": data.get("name", p.stem),
                "url": f"./{p.name}",
                "width": int(shape.get("width", 256)),
                "height": int(shape.get("height", 256))
            })
        except Exception as e:
            logging.warning(f"Manifest: Konnte {p} nicht lesen: {e}")
    out_path = models_dir / "manifest.json"
    with out_path.open("w", encoding="utf-8") as f:
        json.dump(manifest, f, ensure_ascii=False, indent=2)
    logging.info(f"Manifest aktualisiert: {out_path} ({len(files)} Einträge)")
    return out_path

def build_model(width: int, dtype: torch.dtype) -> nn.Sequential:
    m = nn.Sequential(
        nn.Linear(2, width, bias=True, dtype=dtype),
        nn.Tanh(),
        nn.Linear(width, width, bias=True, dtype=dtype),
        nn.Tanh(),
        nn.Linear(width, width, bias=True, dtype=dtype),
        nn.Tanh(),
        nn.Linear(width, width, bias=True, dtype=dtype),
        nn.Tanh(),
        nn.Linear(width, 3, bias=True, dtype=dtype),
    )
    return m

def checkpoint_dir_for(project: str) -> Path:
    d = OUTPUT_DIR / project / 'checkpoints'
    d.mkdir(parents=True, exist_ok=True)
    return d

def save_checkpoint(project: str, step: int, model: nn.Module, optimizer: torch.optim.Optimizer,
                    losses: List[float], ref_shape: Tuple[int,int,int]) -> Path:
    ckpt_dir = checkpoint_dir_for(project)
    ckpt_path = ckpt_dir / f'ckpt_step{step:07d}.pt'
    torch.save({
        "step": step,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "losses": losses[-50_000:],  # nicht unendlich wachsen lassen
        "ref_shape": tuple(int(x) for x in ref_shape)
    }, ckpt_path)
    # "latest" schreiben
    latest_path = ckpt_dir / 'latest.pt'
    try:
        latest_path.unlink(missing_ok=True)
    except Exception:
        pass
    ckpt_path.replace(ckpt_path)  # no-op (placeholder für konsistenz)
    torch.save({
        "step": step,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "losses": losses[-50_000:],
        "ref_shape": tuple(int(x) for x in ref_shape)
    }, latest_path)
    logging.info(f"Checkpoint gespeichert: {ckpt_path.name}")
    return ckpt_path

def load_latest_checkpoint(project: str, model: nn.Module, optimizer: Optional[torch.optim.Optimizer]=None
                           ) -> Tuple[int, List[float], Optional[Tuple[int,int,int]]]:
    ckpt_dir = checkpoint_dir_for(project)
    latest = ckpt_dir / 'latest.pt'
    start_step = 0
    losses: List[float] = []
    ref_shape: Optional[Tuple[int,int,int]] = None
    if latest.exists():
        blob = torch.load(latest, map_location='cpu')
        model.load_state_dict(blob["model_state"])
        if optimizer is not None and "optimizer_state" in blob:
            try:
                optimizer.load_state_dict(blob["optimizer_state"])
            except Exception as e:
                logging.warning(f"Optimizer-State konnte nicht geladen werden (weiter mit frischem Optimizer): {e}")
        start_step = int(blob.get("step", 0))
        losses = list(blob.get("losses", []))
        rs = blob.get("ref_shape")
        if rs is not None:
            ref_shape = (int(rs[0]), int(rs[1]), int(rs[2]))
        logging.info(f"Resume von Step {start_step} (Checkpoint gefunden).")
    return start_step, losses, ref_shape

# ========= Training =========

def train(project: str, n: int, frame: int, threshold: float, model: torch.nn.Module,
          optimizer: torch.optim.Optimizer, dtype: torch.dtype, device: torch.device,
          start_step: int = 0, resume_losses: Optional[List[float]] = None
          ) -> Tuple[int, Tuple[int,int,int], List[float], int]:
    """
    Gibt zurück: (frame, ref_shape, losses, last_step)
    """
    im = read_and_resize_image(project, SIZE, IMAGE_DIR)
    save_image(im, OUTPUT_DIR / project / 'input.png')

    ref_y = torch.tensor(np.array([i for j in im for i in j]), dtype=dtype, device=device)
    ref_shape = (int(im.shape[0]), int(im.shape[1]), 3)
    logging.info(f"ref_shape={ref_shape}")

    ref_grid = np.mgrid[0:ref_shape[0], 0:ref_shape[1]]
    ref_x = torch.hstack([
        torch.tensor(np.array([[i / (ref_shape[0] - 1)] for i in ref_grid[0].flatten()]), dtype=dtype, device=device),
        torch.tensor(np.array([[i / (ref_shape[1] - 1)] for i in ref_grid[1].flatten()]), dtype=dtype, device=device),
    ])

    fig = plt.figure(figsize=(1920 / DPI, 1080 / DPI), dpi=DPI)
    fig.canvas.draw()

    mse = nn.MSELoss(reduction='mean')
    model.train()

    # Verlaufswerte (fortsetzen, falls vorhanden)
    losses: List[float] = resume_losses[:] if resume_losses else []
    best = float(min(losses)) if losses else float('inf')

    # Fortschrittsbalken von start_step bis n
    with logging_redirect_tqdm(), tqdm(
        total=n, initial=start_step,
        desc=f"train:{project}",
        unit="step",
        dynamic_ncols=True,
        leave=True
    ) as pbar:

        def closure():
            optimizer.zero_grad(set_to_none=True)
            err = mse(model.forward(ref_x), ref_y)
            err.backward()
            return err

        last_saved_at = (start_step // CHECKPOINT_EVERY) * CHECKPOINT_EVERY

        for i in range(start_step, n):
            loss_tensor: torch.Tensor = optimizer.step(closure)
            loss_val = float(loss_tensor.detach().cpu().numpy())
            losses.append(loss_val)
            if loss_val < best:
                best = loss_val

            pbar.set_postfix(loss=f"{loss_val:.4f}", best=f"{best:.4f}")
            pbar.update(1)

            # Frühstopp prüfen
            change = abs(np.min(losses[-2_000:-1_000]) - np.min(losses[-1_000:])) if i > 2_000 else np.inf
            if i > 2_000 and np.min(losses) < threshold and change < 1:
                pbar.set_postfix(loss=f"{loss_val:.4f}", best=f"{best:.4f}", note="early-stop")
                last_step = i
                break

            # Sparsame Visualisierung
            if i % 10 == 0:
                art = copy.deepcopy(model).to(device).eval()
                weights, biases = [], []
                for name, param in art.named_parameters():
                    if 'weight' in name:  weights.append(param.detach())
                    if 'bias'   in name:  biases.append(param.detach())
                weights, biases = artsy(weights, biases)
                for name, param in art.named_parameters():
                    if 'weight' in name: param.data = nn.parameter.Parameter(weights.pop(0))
                    if 'bias'   in name: param.data = nn.parameter.Parameter(biases.pop(0))
                with torch.inference_mode():
                    a = art.forward(ref_x).detach().cpu().numpy().reshape(ref_shape)
                    base = model.forward(ref_x).detach().cpu().numpy().reshape(ref_shape)
                save_frame(fig, i, model, base, a, losses)
                save_fig(fig, OUTPUT_DIR / project / 'frames' / f'frame_{frame:06d}.png')
                save_art(a, OUTPUT_DIR / project / 'art' / f'frame_{frame:06d}.png')
                frame += 1
                # Cleanup
                del art, weights, biases, a, base
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

            # === Checkpoint + JSON + Manifest alle 10k Steps ===
            if (i - last_saved_at) >= CHECKPOINT_EVERY or i == n - 1:
                save_checkpoint(project, i, model, optimizer, losses, ref_shape)
                save_model_json(model, project, ref_shape, MODELS_DIR)
                write_manifest(MODELS_DIR)
                last_saved_at = i
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()

        else:
            # lief bis zum Ende ohne break
            last_step = n - 1

    # Low-res Output (Train-Auflösung)
    model.eval()
    with torch.inference_mode():
        base = model.forward(ref_x).detach().cpu().numpy().reshape(ref_shape)
    save_image(base, OUTPUT_DIR / project / 'output.png')

    # Hi-Res Eval (8×) — gekachelt
    eval_H, eval_W = 8 * ref_shape[0], 8 * ref_shape[1]
    base_hi = render_image_tiled(model, eval_H, eval_W, device, tile=256)
    save_image(base_hi, OUTPUT_DIR / project / 'eval.png')
    del base_hi
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    art_hi = render_art_tiled(model, eval_H, eval_W, device, tile=256)
    save_art(art_hi, OUTPUT_DIR / project / 'art.png')
    del art_hi
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Große Tensoren freigeben
    del im, ref_x, ref_y, ref_grid, fig, base
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return frame, ref_shape, losses, last_step

# ========= Main =========

def main() -> None:
    if not torch.cuda.is_available():
        logging.warning("CUDA/HIP nicht verfügbar – wechsle auf CPU.")
        device = torch.device('cpu')
    else:
        device = torch.device('cuda')  # funktioniert auch mit HIP-Backend

    dtype = torch.float32
    layer = SIZE

    if not PROJECTS:
        raise SystemExit(
            f"Keine Projekte gefunden. Lege Bilder in {IMAGE_DIR} ab "
            f"(erlaubte Endungen: {', '.join(ALLOWED_EXTS)}), oder deaktiviere AUTO_DISCOVER_PROJECTS und setze PROJECTS manuell."
        )

    for project in PROJECTS:
        model = build_model(layer, dtype).to(device)
        optimizer = torch.optim.Adam(model.parameters())
        start_step, resume_losses, resume_shape = load_latest_checkpoint(project, model, optimizer)

        n = 1_000_000
        threshold = 100
        frame = 1

        # ---- Training (mit Resume) ----
        frame, ref_shape, losses, last_step = train(
            project, n, frame, threshold, model, optimizer, dtype, device,
            start_step=start_step, resume_losses=resume_losses
        )

        # ---- Final speichern ----
        pt_path = OUTPUT_DIR / project / 'model.pt'
        pt_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save(model.state_dict(), pt_path)
        logging.info(f"state_dict gespeichert: {pt_path}")

        save_model_json(model, project, ref_shape, MODELS_DIR)
        write_manifest(MODELS_DIR)

        # Cleanup pro Projekt
        del model, optimizer, losses
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

if __name__ == '__main__':
    logging.basicConfig(
        format='%(asctime)s %(module)s[%(levelname)s]: %(message)s',
        handlers=[logging.StreamHandler(sys.stdout)],
        encoding='utf-8',
        level=logging.INFO
    )
    torch.manual_seed(42)
    random.seed(42)
    np.random.seed(42)

    try:
        main()
        logging.info('EXIT -- Success')
    except KeyboardInterrupt:
        logging.info('EXIT -- Abort')


2025-08-17 10:42:54,212 3469687410[INFO]: Sequential(
  (0): Linear(in_features=2, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Tanh()
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): Tanh()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): Tanh()
  (8): Linear(in_features=256, out_features=3, bias=True)
)
2025-08-17 10:42:55,326 3469687410[INFO]: Loading /home/ich/Schreibtisch/Ki/artevis/src/images/mond.jpg
2025-08-17 10:42:55,367 3469687410[INFO]: ref_shape=(246, 256, 3)


/tmp/ipykernel_1173881/3469687410.py:73: UserWarning: cmr10 font should ideally be used with mathtext, set axes.formatter.use_mathtext to True
  sub = fig.add_subplot()
train:mond:   0%|          | 1/1000000 [00:00<62:51:08,  4.42step/s, best=11746.0166, loss=11746.0166]/tmp/ipykernel_1173881/3469687410.py:111: UserWarning: cmr10 font should ideally be used with mathtext, set axes.formatter.use_mathtext to True
  sub = fig.add_subplot(2, 1, 2)
/tmp/ipykernel_1173881/3469687410.py:115: UserWarning: cmr10 font should ideally be used with mathtext, set axes.formatter.use_mathtext to True
  sub = fig.add_subplot(2, len(weights) + 2, len(weights) + 1)
/tmp/ipykernel_1173881/3469687410.py:118: UserWarning: cmr10 font should ideally be used with mathtext, set axes.formatter.use_mathtext to True
  sub = fig.add_subplot(2, len(weights) + 2, len(weights) + 2)
/tmp/ipykernel_1173881/3469687410.py:137: UserWarning: cmr10 font should ideally be used with mathtext, set axes.formatter.use_mathtext to

2025-08-17 11:33:38,465 3469687410[INFO]: state_dict gespeichert: /home/ich/Schreibtisch/Ki/artevis/src/output/mond/model.pt
2025-08-17 11:33:38,570 3469687410[INFO]: Model-JSON geschrieben: /home/ich/Schreibtisch/Ki/artevis/src/webroot/models/mond.json
2025-08-17 11:33:38,572 3469687410[INFO]: Manifest aktualisiert: /home/ich/Schreibtisch/Ki/artevis/src/webroot/models/manifest.json (3 Einträge)
2025-08-17 11:33:38,575 3469687410[INFO]: Sequential(
  (0): Linear(in_features=2, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Tanh()
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): Tanh()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): Tanh()
  (8): Linear(in_features=256, out_features=3, bias=True)
)
2025-08-17 11:33:38,576 3469687410[INFO]: Loading /home/ich/Schreibtisch/Ki/artevis/src/images/nebelmeer.jpg
2025-08-17 11:33:38,611 3469687410[INFO]: ref_shape=(256, 202, 3)


train:nebelmeer:   4%|▎         | 36543/1000000 [39:44<17:27:53, 15.32step/s, best=40.9781, loss=41.2288, note=early-stop]


2025-08-17 12:13:25,875 3469687410[INFO]: state_dict gespeichert: /home/ich/Schreibtisch/Ki/artevis/src/output/nebelmeer/model.pt
2025-08-17 12:13:25,987 3469687410[INFO]: Model-JSON geschrieben: /home/ich/Schreibtisch/Ki/artevis/src/webroot/models/nebelmeer.json
2025-08-17 12:13:25,988 3469687410[INFO]: Manifest aktualisiert: /home/ich/Schreibtisch/Ki/artevis/src/webroot/models/manifest.json (4 Einträge)
2025-08-17 12:13:25,992 3469687410[INFO]: Sequential(
  (0): Linear(in_features=2, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Tanh()
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): Tanh()
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): Tanh()
  (8): Linear(in_features=256, out_features=3, bias=True)
)
2025-08-17 12:13:25,993 3469687410[INFO]: Loading /home/ich/Schreibtisch/Ki/artevis/src/images/schrei.jpg
2025-08-17 12:13:26,035 3469687410[INFO]: ref_shape=(256, 205, 3)


train:schrei:  28%|██▊       | 283406/1000000 [8:16:18<20:54:54,  9.52step/s, best=125.5102, loss=128.0563]


2025-08-17 20:29:44,289 3469687410[INFO]: EXIT -- Abort
